In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
import multiprocessing as mp
import os
import cv2
import pickle
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
from tqdm import tnrange
from sklearn.manifold import TSNE

#Check GPU support, please do activate GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def plot_training_progress(losses, accuracies):
    """
    Plots training loss and accuracy over epochs.

    Args:
        losses (list): List of epoch losses.
        accuracies (list): List of epoch accuracies.
    """
    epochs = range(1, len(losses) + 1)

    plt.figure(figsize=(12, 5))

    # Create a plot for the loss
    ax1 = plt.gca()  # Get current axis
    ax1.plot(epochs, losses, label='Loss', color='blue', marker='o')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Loss', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax1.grid()

    # Create a second y-axis for accuracy
    ax2 = ax1.twinx()  # Create a twin axis sharing the same x-axis
    ax2.plot(epochs, accuracies, label='Accuracy', color='orange', marker='o')
    ax2.set_ylabel('Accuracy', color='orange')
    ax2.tick_params(axis='y', labelcolor='orange')

    # Combine legends
    lines, labels = ax1.get_legend_handles_labels()  # Get the loss labels
    lines2, labels2 = ax2.get_legend_handles_labels()  # Get the accuracy labels
    ax1.legend(lines + lines2, labels + labels2)  # Combine both legends

    plt.tight_layout()
    plt.show()
def extract_sample(n_way, n_support, n_query, datax, datay):
  """
  Picks random sample of size n_support+n_querry, for n_way classes
  Args:
      n_way (int): number of classes in a classification task
      n_support (int): number of labeled examples per class in the support set
      n_query (int): number of labeled examples per class in the query set
      datax (np.array): dataset of images
      datay (np.array): dataset of labels
  Returns:
      (dict) of:
        (torch.Tensor): sample of images. Size (n_way, n_support+n_query, (dim))
        (int): n_way
        (int): n_support
        (int): n_query
  """
  sample = []
  #extract random classes
  K = np.random.choice(np.unique(datay), n_way, replace=False)
  #extract samples from each claass
  for cls in K:
    datax_cls = datax[datay == cls]
    perm = np.random.permutation(datax_cls)
    sample_cls = perm[:(n_support+n_query)]
    sample.append(sample_cls)
  sample = np.array(sample)
  sample = torch.from_numpy(sample).float()
  sample = sample.permute(0,1,4,2,3)
  return({
      'images': sample,
      'n_way': n_way,
      'n_support': n_support,
      'n_query': n_query
      })
class Flatten(nn.Module):
    def __init__(self):
        super(Flatten, self).__init__()

    def forward(self, x):
        return x.view(x.size(0), -1)

def load_protonet_conv(**kwargs):
    """
    Loads the prototypical network model
    Arg:
        x_dim (tuple): dimension of input image
        hid_dim (int): dimension of hidden layers in conv blocks
        z_dim (int): dimension of embedded image
    Returns:
        Model (Class ProtoNet)
    """
    x_dim = kwargs['x_dim']
    hid_dim = kwargs['hid_dim']
    z_dim = kwargs['z_dim']

    def conv_block(in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=2, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

    encoder = nn.Sequential(
        conv_block(x_dim[0], hid_dim),
        conv_block(hid_dim, hid_dim),
        conv_block(hid_dim, hid_dim),
        conv_block(hid_dim, z_dim),
        Flatten()
    )

    # Initialize weights using Kaiming Normal initialization
    for m in encoder.modules():
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)
        elif isinstance(m, nn.Linear):  # If there are linear layers
            nn.init.kaiming_normal_(m.weight, mode='fan_in', nonlinearity='relu')
            if m.bias is not None:
                nn.init.zeros_(m.bias)

    return ProtoNet(encoder)
class ProtoNet(nn.Module):
  def __init__(self, encoder):
    """
    Args:
        encoder : CNN encoding the images in sample
        n_way (int): number of classes in a classification task
        n_support (int): number of labeled examples per class in the support set
        n_query (int): number of labeled examples per class in the query set
    """
    super(ProtoNet, self).__init__()
    self.encoder = encoder.to(device)

  def set_forward_loss(self, sample):
    """
    Computes loss, accuracy and output for classification task
    Args:
        sample (torch.Tensor): shape (n_way, n_support+n_query, (dim))
    Returns:
        torch.Tensor: shape(2), loss, accuracy and y_hat
    """
    sample_images = sample['images'].to(device)
    n_way = sample['n_way']
    n_support = sample['n_support']
    n_query = sample['n_query']

    x_support = sample_images[:, :n_support]
    x_query = sample_images[:, n_support:]

    #target indices are 0 ... n_way-1
    target_inds = torch.arange(0, n_way).view(n_way, 1, 1).expand(n_way, n_query, 1).long()
    target_inds = Variable(target_inds, requires_grad=False)
    target_inds = target_inds.to(device)

    #encode images of the support and the query set
    x = torch.cat([x_support.contiguous().view(n_way * n_support, *x_support.size()[2:]),
                   x_query.contiguous().view(n_way * n_query, *x_query.size()[2:])], 0)

    z = self.encoder.forward(x)
    z_dim = z.size(-1) #usually 64
    # Extract support and query samples
    z_samples = z[:n_way * n_support].view(n_way, n_support, z_dim)
    z_query = z[n_way * n_support:]

    # Ensure gradients are tracked
    z_samples.requires_grad_(True)
    z_query.requires_grad_(True)

    #----------------------------Making of prototypes---------------------------
    z_proto = z_samples.mean(dim=1)
    #-----------------calculationg distance from each prototypes----------------
    dists = euclidean_dist(z_query,z_proto)
    #--------------compute probabilities,loss and accuracy----------------------
    log_p_y = F.log_softmax(-dists, dim=1).view(n_way, n_query, -1)
    loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
    _, y_hat = log_p_y.max(2)
    acc_val = torch.eq(y_hat, target_inds.squeeze()).float().mean()


    return loss_val, {
        'loss': loss_val.item(),
        'acc': acc_val.item(),
        'y_hat': y_hat
        }
def euclidean_dist(x, y):
  """
  Computes euclidean distance btw x and y
  Args:
      x (torch.Tensor): shape (n, d). n usually n_way*n_query
      y (torch.Tensor): shape (m, d). m usually n_way
  Returns:
      torch.Tensor: shape(n, m). For each query, the distances to each centroid
  """
  n = x.size(0)
  m = y.size(0)
  d = x.size(1)
  assert d == y.size(1)

  x = x.unsqueeze(1).expand(n, m, d)
  y = y.unsqueeze(0).expand(n, m, d)

  return torch.pow(x - y, 2).sum(2)
def train(model, optimizer, train_x, train_y, n_way, n_support, n_query, max_epoch, epoch_size):
    """
    Trains the protonet
    Args:
        model
        optimizer
        train_x (np.array): images of training set
        train_y(np.array): labels of training set
        n_way (int): number of classes in a classification task
        n_support (int): number of labeled examples per class in the support set
        n_query (int): number of labeled examples per class in the query set
        max_epoch (int): max epochs to train on
        epoch_size (int): episodes per epoch
    """
    # Divide the learning rate by 2 at each epoch, as suggested in the paper
    scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5, last_epoch=-1)
    epoch = 0  # Epochs done so far
    stop = False  # Status to know when to stop

    # Lists to store loss and accuracy for plotting
    epoch_losses = []
    epoch_accuracies = []

    while epoch < max_epoch and not stop:
        running_loss = 0.0
        running_acc = 0.0

        for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch + 1)):
            sample = extract_sample(n_way, n_support, n_query, train_x, train_y)
            optimizer.zero_grad()
            loss, output = model.set_forward_loss(sample)
            running_loss += output['loss']
            running_acc += output['acc']
            loss.backward()
            optimizer.step()

        epoch_loss = running_loss / epoch_size
        epoch_acc = running_acc / epoch_size
        print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoch + 1, epoch_loss, epoch_acc))

        # Store the values for plotting
        epoch_losses.append(epoch_loss)
        epoch_accuracies.append(epoch_acc)

        epoch += 1
        scheduler.step()

    # Call the plotting function after training
    #plot_training_progress(epoch_losses, epoch_accuracies)

import matplotlib.cm as cm

def test(model, test_x, test_y, n_way, n_support, n_query, test_episode, visualize=True):
    """
    Runs a few‐shot test loop, optionally visualizing the prototype and support embeddings
    with t-SNE each episode.

    Returns:
        float: mean accuracy over test_episode runs
    """
    model.eval()
    all_losses, all_accs = [], []

    for ep in range(test_episode):
        # Draw a new episode
        sample = extract_sample(n_way, n_support, n_query, test_x, test_y)

        # Compute loss and accuracy
        loss_val, out = model.set_forward_loss(sample)
        all_losses.append(out['loss'])
        all_accs.append(out['acc'])

        if visualize:
            # Prepare support + prototype embeddings
            imgs = sample['images'].to(device)
            x_support = imgs[:, :n_support]                                      # (n_way, n_support, C, H, W)

            # flatten for encoder
            x_sup_flat = x_support.contiguous().view(n_way * n_support, *x_support.size()[2:])

            with torch.no_grad():
                z_sup_all = model.encoder(x_sup_flat)                             # (n_way*n_support, z_dim)

            z_dim = z_sup_all.size(-1)
            # reshape to separate classes
            z_sup = z_sup_all.view(n_way, n_support, z_dim)                      # (n_way, n_support, z_dim)
            z_proto = z_sup.mean(dim=1)                                          # (n_way, z_dim)

            # Stack prototypes first, then all support points
            embeddings = torch.cat([z_proto, z_sup_all], dim=0).cpu().numpy()    # ((n_way + n_way*n_support), z_dim)

            # Build labels: 0..n_way-1 for prototypes, then repeated class‐ids for supports
            proto_labels   = np.arange(n_way)
            support_labels = np.repeat(np.arange(n_way), n_support)
            labels = np.concatenate([proto_labels, support_labels])

            num_samples = embeddings.shape[0]
            tsne_perplexity = min(5, (num_samples - 1) // 3)

            # Run t-SNE
            coords = TSNE(n_components=2, perplexity=tsne_perplexity).fit_transform(embeddings)

            # Plot
            plt.figure(figsize=(6, 6),dpi=300)
            colors = [
    (0.8, 0.1, 0.1),   # Red
    (0.1, 0.6, 0.1),   # Green
    (0.1, 0.2, 0.6)    # Dark Blue
]


            # 1) prototypes as big X’s
            for cls in range(n_way):
                plt.scatter(
                    coords[cls, 0], coords[cls, 1],
                    color=colors[cls], marker='X', s=200,
                    edgecolor='black', label=f'Prototype {cls}'
                )

            # 2) support samples as small dots
            start_sup = n_way
            for cls in range(n_way):
                idxs = start_sup + cls * n_support
                plt.scatter(
                    coords[idxs: idxs + n_support, 0],
                    coords[idxs: idxs + n_support, 1],
                    color=colors[cls], marker='o', s=100,
                    alpha=0.7, label=f'Class {cls} support'
                )

            plt.legend(loc='best', fontsize='small')
            #plt.title(f't-SNE Support – Episode {ep + 1}')
            plt.grid(True)
            plt.show()

    mean_loss = np.mean(all_losses)
    mean_acc  = np.mean(all_accs)
    print(f"Test results  -- Loss: {mean_loss:.4f}  Acc: {mean_acc:.4f}")
    return mean_acc




#-----------Main function-----------------------------------------------------

n_way = 3
n_support = 10
n_query = 5
epochs_to_run = [20]
epoch_size = 2000
test_episode = 600
#-------------------------set seeeds-----------------------------
np_seed = 0 # Set the seed for reproducibility
torch_seed = 0
Size=84
# Fix all seeds
np.random.seed(np_seed)
torch.manual_seed(torch_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(torch_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load pickle files
PATH = '/content/gdrive/MyDrive/Refined-Pickle-Files/Refined-Pickle-Files/Pickle-'+str(Size)+'_'+str(Size)+'/'
Data='BLOOD-random'

with open(PATH + Data + "_trainx.pkl", "rb") as f:
    trainx = pickle.load(f)
with open(PATH + Data + "_trainy.pkl", "rb") as f:
    trainy = pickle.load(f)
with open(PATH + Data + "_testx.pkl", "rb") as f:
    testx = pickle.load(f)
with open(PATH + Data + "_testy.pkl", "rb") as f:
    testy = pickle.load(f)

# Function to track the highest accuracy
def get_accuracy(model, testx, testy, n_way, n_support, n_query, test_episode):
    # Assuming this function is available in your framework
    return test(model, testx, testy, n_way, n_support, n_query, test_episode)

best_accuracy = 0
best_epoch = 0

# Training and testing for specified epochs
for epoch in epochs_to_run:
    # Initialize model and optimizer
    model = load_protonet_conv(x_dim=(3, Size, Size), hid_dim=64, z_dim=64)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    print(f"-----------------------Training for {epoch} epochs-----------------------")
    train(model, optimizer, trainx, trainy, n_way, n_support, n_query, epoch, epoch_size)

    print(f"-----------------------Testing after {epoch} epochs-----------------------")
    accuracy = get_accuracy(model, testx, testy, n_way, n_support, n_query, test_episode)

    print(f"Accuracy after {epoch} epochs: {accuracy}")

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_epoch = epoch

print(f"\nHighest accuracy: {best_accuracy} achieved at epoch: {best_epoch}")


Output hidden; open in https://colab.research.google.com to view.